# Pulling data from a Data Model and generate OCEL JSON file

In [1]:
import pandas as pd
import json
from typing import Dict, Optional, Any
from io import BytesIO
from pycelonis import get_celonis
from pycelonis.pql.pql import PQL, PQLColumn
from pycelonis.service.integration.service import ExportType
from pycelonis.ems.data_integration.data_pool import DataModel

In [2]:
#https://github.com/fit-alessandro-berti/pm4py-core/blob/celonis-integration/pm4py/algo/celonis/retrieval/dm_retrieval.py
def apply(data_model: DataModel, parameters: Optional[Dict[Any, Any]] = None) -> Dict[str, pd.DataFrame]:
    """
    Retrieves the tables of a data model in Celonis

    Parameters
    --------------
    data_model
        Celonis data model

    Returns
    -------------
    dct
        Dictionary associating each name to a dataframe extracted from the data model
    """
    if parameters is None:
        parameters = {}

    dct = {}
    tables = data_model.get_tables()
    for table in tables:
        table2 = data_model.get_table(table.id)
        columns = table2.get_columns()
        buffer = BytesIO()
        excluded_columns = set()
        for i in range(2):
            try:
                pql = PQL()
                for col in columns:
                    if not col.name.startswith("_"):
                        fname = "\"" + table2.name + "\".\"" + col.name + "\""
                        if fname not in excluded_columns:
                            pql.add(PQLColumn(query=fname, name=col.name))
                dexp = data_model.create_data_export(query=pql, export_type=ExportType.PARQUET)
                dexp.wait_for_execution()
                chunks = dexp.get_chunks()
                for chunk in chunks:
                    buffer.write(chunk.read())
                df = pd.read_parquet(buffer)
                dct[table.name] = df
                break
            except Exception as e:
                e = str(e)
                cols = e.split(" is missing")
                for c in cols:
                    c = c.split("Column ")[-1]
                    excluded_columns.add(c)
    return dct

# Extract all tables from a data model in Celonis

In [3]:
celonis = get_celonis()

[2023-06-27 10:52:53,324] INFO: No `base_url` given. Using environment variable 'CELONIS_URL'
[2023-06-27 10:52:53,324] INFO: No `api_token` given. Using environment variable 'CELONIS_API_TOKEN'


[2023-06-27 10:52:53,371] WARNING: Your PyCelonis Version 2.2.0 is outdated (Newest Version: 2.3.0). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis pycelonis_core --upgrade
[2023-06-27 10:52:53,415] WARNING: KeyType is not set. Defaulted to 'APP_KEY'.


[2023-06-27 10:52:53,424] INFO: Initial connect successful! PyCelonis Version: 2.2.0
[2023-06-27 10:52:53,447] INFO: `package-manager` permissions: ['EDIT_ALL_SPACES', 'MANAGE_PERMISSIONS', 'CREATE_SPACE', 'DELETE_ALL_SPACES']
[2023-06-27 10:52:53,448] INFO: `workflows` permissions: []
[2023-06-27 10:52:53,448] INFO: `task-mining` permissions: []
[2023-06-27 10:52:53,449] INFO: `action-engine` permissions: ['CREATE_PROJECTS', 'MANAGE_SKILLS', 'ACCESS_ALL_PROJECTS', 'MY_INBOX']
[2023-06-27 10:52:53,450] INFO: `team` permissions: ['MANAGE_AUDIT_LOGS', 'MANAGE_SSO_SETTINGS', 'USE_AUDIT_LOGS_API', 'MANAGE_ADOPTION_VIEWS', 'MANAGE_GENERAL_SETTINGS', 'MANAGE_GROUPS', 'MANAGE_APPLICATIONS', 'MANAGE_LOGIN_HISTORY', 'MANAGE_LICENSE_SETTINGS', 'USE_LOGIN_HISTORY_API', 'MANAGE_MEMBERS', 'MANAGE_UPLINK_INTEGRATIONS', 'MANAGE_PERMISSIONS', 'MANAGE_ADMIN_NOTIFICATIONS', 'IMPORT_MEMBERS', 'MANAGE_MEMBER_LOCKING_POLICY']
[2023-06-27 10:52:53,450] INFO: `process-repository` permissions: []
[2023-06-27 

In [4]:
#celonis.data_integration.get_data_pools()

In [5]:
data_pool_id = '8cd1f9f8-668a-441f-8e46-47d5fef97156'
data_pool = celonis.data_integration.get_data_pool(data_pool_id)

In [6]:
data_pool.get_data_models()

[
	DataModel(id='2abf9d9d-ce35-4f60-b9e9-463e5aad9751', name='Oracle EBS P2P OCEL', pool_id='8cd1f9f8-668a-441f-8e46-47d5fef97156'),
	DataModel(id='d974d6c1-6866-407f-97df-d7320a3fa9c2', name='Oracle EBS P2P OCEL (New setup)', pool_id='8cd1f9f8-668a-441f-8e46-47d5fef97156')
]

In [7]:
data_model_id = 'd974d6c1-6866-407f-97df-d7320a3fa9c2'

In [8]:
data_model = data_pool.get_data_model(data_model_id)
data_model

DataModel(id='d974d6c1-6866-407f-97df-d7320a3fa9c2', name='Oracle EBS P2P OCEL (New setup)', pool_id='8cd1f9f8-668a-441f-8e46-47d5fef97156')

In [9]:
#Call the function with the data model to get all the tables extracted
dataframe_dict = apply(data_model)

[2023-06-27 10:52:53,907] INFO: Successfully created data export with id 'c97fd2fa-c875-4139-a532-47be16596814'
[2023-06-27 10:52:53,908] INFO: Wait for execution of data export with id 'c97fd2fa-c875-4139-a532-47be16596814'


0it [00:00, ?it/s]

[2023-06-27 10:52:55,997] INFO: Export result chunks for data export with id 'c97fd2fa-c875-4139-a532-47be16596814'
[2023-06-27 10:52:58,095] INFO: Successfully created data export with id 'c34b38b4-b489-4edb-88db-c6ee14cc9fd8'
[2023-06-27 10:52:58,096] INFO: Wait for execution of data export with id 'c34b38b4-b489-4edb-88db-c6ee14cc9fd8'


0it [00:00, ?it/s]

[2023-06-27 10:53:00,175] INFO: Export result chunks for data export with id 'c34b38b4-b489-4edb-88db-c6ee14cc9fd8'
[2023-06-27 10:53:00,457] INFO: Successfully created data export with id '6e3496b8-aa27-471b-ae38-b8a54c54089a'
[2023-06-27 10:53:00,458] INFO: Wait for execution of data export with id '6e3496b8-aa27-471b-ae38-b8a54c54089a'


...

[2023-06-27 10:53:42,592] INFO: Export result chunks for data export with id 'd2dc54fa-bfc6-4930-9f43-550574cc93aa'
[2023-06-27 10:53:42,815] INFO: Successfully created data export with id 'd94f50fb-5d8f-4c2c-bf9e-1b651f408c8a'
[2023-06-27 10:53:42,815] INFO: Wait for execution of data export with id 'd94f50fb-5d8f-4c2c-bf9e-1b651f408c8a'


0it [00:00, ?it/s]

[2023-06-27 10:53:44,879] INFO: Export result chunks for data export with id 'd94f50fb-5d8f-4c2c-bf9e-1b651f408c8a'
[2023-06-27 10:53:45,099] INFO: Successfully created data export with id 'bebc4daf-6bb7-4127-9ece-3a1472a60a9b'
[2023-06-27 10:53:45,100] INFO: Wait for execution of data export with id 'bebc4daf-6bb7-4127-9ece-3a1472a60a9b'


0it [00:00, ?it/s]

[2023-06-27 10:53:47,167] INFO: Export result chunks for data export with id 'bebc4daf-6bb7-4127-9ece-3a1472a60a9b'


In [10]:
# the tables exported into dataframes
print(dataframe_dict.keys())

dict_keys(['_LINK_EVENT_ID_OBJECT_ID', '_PL_PO_LINES_ALL', '_CA_PO_HEADERS_ALL_CONTRACTS', '_PO_PO_HEADERS_ALL', '_PR_PO_REQUISITION_HEADERS_ALL', '_SU_AP_SUPPLIERS', '_AI_AP_INVOICES_ALL', '_GR_RCV_SHIPMENT_HEADERS', '_IH_AP_HOLDS_ALL', '_IP_AP_INVOICE_PAYMENTS_ALL', '_BA_IBY_EXT_BANK_ACCOUNTS', '_EVENTLOG', '_LINK_AI_IH', '_LINK_AI_IP', '_LINK_BA_IP', '_LINK_BA_SU', '_LINK_CA_PO', '_LINK_CA_PR', '_LINK_CA_SU', '_LINK_GR_AI', '_LINK_PO_AI', '_LINK_PO_GR', '_LINK_PO_PL', '_LINK_PR_PO', '_LINK_SU_AI', '_LINK_SU_GR', '_LINK_SU_PO'])


In [11]:
# # drop some of the dataframes to keep size of the sample ocel file limited
# keys_to_drop = ['_PL_PO_LINES_ALL', '_CA_PO_HEADERS_ALL_CONTRACTS', '_IH_AP_HOLDS_ALL', '_IP_AP_INVOICE_PAYMENTS_ALL', '_GR_RCV_SHIPMENT_HEADERS']

# for key in list(dataframe_dict.keys()):  # use list() to create a copy of keys
#     if key.startswith('_LINK') or key in keys_to_drop:
#         # Don't remove the _LINK_EVENT_ID_OBJECT_ID dataframe
#         if key != '_LINK_EVENT_ID_OBJECT_ID':
#             dataframe_dict.pop(key)

# Use the exported dataframes to generate the OCEL JSON file

In [12]:
# get the unique attribute names for the objects
# Initialize an empty set to store unique column names
attribute_names = set()

# Iterate over each DataFrame in the dictionary
for key, df in dataframe_dict.items():
    # If the key does not start with '_LINK'
    if not key.startswith('_LINK'):
        # Add the column names to the set
        attribute_names.update(df.columns)

# Convert the set to a list
attribute_names_list = list(attribute_names)

# attribute_names_list is a list containing the unique column names
#attribute_names_list

In [13]:
# define object type mapping dictionary
object_type_mapping_dict = {
    'IH': 'InvoiceHold', 
    'AI': 'APInvoice', 
    'IP': 'InvoicePayment', 
    'SU': 'Supplier', 
    'BA': 'BankAccount', 
    'PO': 'PurchaseOrder',
    'PR': 'PurchaseRequisition', 
    'CA': 'ContractAgreement', 
    'PL': 'PurchaseOrderLine', 
    'GR': 'GoodsReceipt'
}

In [14]:
# Get the unique object types
# Get the _LINK_EVENT_ID_OBJECT_ID DataFrame from the dictionary
df_link = dataframe_dict['_LINK_EVENT_ID_OBJECT_ID']

# Ensure 'OBJECT_ID' and 'EVENT_ID' is a string
df_link['OBJECT_ID'] = df_link['OBJECT_ID'].astype(str)
df_link['EVENT_ID'] = df_link['EVENT_ID'].astype(str)

#get the unique object types in a list
object_types = df_link['OBJECT_ID'].apply(lambda x: x[:2]).unique()

# Map the unique_values to their full names and convert to list
object_types_list = [object_type_mapping_dict.get(value, value) for value in object_types]

# object_types_list is a list with the unique full names of the object types
#object_types_list

In [15]:
##only 1000 rows for testing purposes
#df_link = df_link.iloc[:1000]

In [16]:
# get all the events
events_df = dataframe_dict['_EVENTLOG']
events_df['EVENT_ID'] = events_df['EVENT_ID'].astype(str)
events_df['EVENTTIME'] = pd.to_datetime(events_df['EVENTTIME'])

In [17]:
##only 1000 rows for testing purposes
#events_df = events_df.iloc[:1000]

In [18]:
# get the object id's per event id in a dictionary
event_id_to_object_ids = df_link.groupby('EVENT_ID')['OBJECT_ID'].apply(list).to_dict()

In [19]:
# Open the file
with open('ocel_oracle_ebs_p2p.jsonocel', 'w') as f:
    # Write the start of the dictionary
    f.write('{')

    # Write the static parts of the JSON data
    f.write('"ocel:global-event": {"ocel:activity": "__INVALID__"},')
    f.write('"ocel:global-object": {"ocel:type": "__INVALID__"},')
    f.write('"ocel:global-log": {"ocel:attribute-names": ' + json.dumps(attribute_names_list) + ', "ocel:object-types": ' + json.dumps(object_types_list) + ', "ocel:version": "1.0", "ocel:ordering": "timestamp"},')
                
            
    # Write the start of the "ocel:events" list
    f.write('"ocel:events": {')
    
    # Exclude columns already used
    attribute_columns = [col for col in events_df.columns if col not in ['EVENT_ID', 'ACTIVITY_EN', 'EVENTTIME', 'PRIMARY_OBJECT_ID','ACTIVITY_NL','ACTIVITY_DETAIL_NL']]
    
    # Iterate over the rows in df_events
    first_event = True
    for _, event_row in events_df.iterrows():
        # Get the objects for this event from event_id_to_object_ids
        objects = event_id_to_object_ids.get(event_row['EVENT_ID'], [])

        # Create attribute map
        attributes = {attr: str(event_row[attr]) for attr in attribute_columns if pd.notnull(event_row[attr])}  # Only include attributes with valid values

        # If this isn't the first event, write a comma to separate it from the previous event
        if not first_event:
            f.write(',')
        else:
            first_event = False

            
        # Create the event data dictionary
        event_data = {
            'ocel:activity': event_row['ACTIVITY_EN'],
            'ocel:timestamp': event_row['EVENTTIME'].isoformat(),
            'ocel:omap': objects,
            'ocel:vmap': attributes
        }

        # Write the event data
        f.write(f'"{event_row["EVENT_ID"]}": ' + json.dumps(event_data))
            
    # Write the end of the "ocel:events" list
    f.write('},')

    # Write the start of the "ocel:objects" list
    f.write('"ocel:objects": {')

    # Iterate over the dataframes in dataframe_dict
    first_object = True
    for df_name, df in dataframe_dict.items():
        if not df_name.startswith('_LINK') and not df_name.startswith('_EVENTLOG'):
            # Iterate over the rows in the dataframe
            for _, row in df.iterrows():
                # Extract type
                object_type_abbr = row['OBJECT_ID'][:2]
                object_type = object_type_mapping_dict.get(object_type_abbr, object_type_abbr)  # map object type abbreviation to full name

                # Create a dictionary for this object
                object_dict = {col: str(row[col]) for col in df.columns if col != 'OBJECT_ID' and pd.notnull(row[col])}

                # If this isn't the first object, write a comma to separate it from the previous object
                if not first_object:
                    f.write(',')
                else:
                    first_object = False
                    
                # Create the object data dictionary
                object_data = {
                    'ocel:type': object_type,
                    'ocel:ovmap': object_dict
                }
                
                # Write the object data
                f.write(f'"{row["OBJECT_ID"]}": ' + json.dumps(object_data))

    # Write the end of the "ocel:objects" list and the end of the dictionary
    f.write('}}')